In [5]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.activations import relu, tanh, sigmoid, softmax
from sklearn.model_selection import train_test_split
import pathlib

In [6]:
crop_data = pd.read_csv('/tmp/Crop_recommendation.csv')
crop_data.drop(['N', 'P', 'K', 'ph'], axis=1, inplace=True)
crop_data.head()

,temperature,humidity,rainfall,label
0,20.879744,82.002744,202.935536,rice
1,21.770462,80.319644,226.655537,rice
2,23.004459,82.320763,263.964248,rice
3,26.491096,80.158363,242.864034,rice
4,20.130175,81.604873,262.717340,rice


# Data Preprocessing

In [7]:
# Take the first eleven labels
first_11_labels = crop_data['label'].unique()[:11]

In [8]:
# Encode the labels to form 0 and 1
def encode_label(data, label):
    if data['label'] == label:
        data['label'] = 1
    else:
        data['label'] = 0
    return data

# Take 200 data from original dataset
def take_200(data, label):
    
    # Data for particular label
    specific_data = data[data['label'] == label]
    
    n = len(specific_data)
    
    # Data for beside particular label
    beside_data = data[data['label'] != label].sample(n)
    
    # Combine specific_data and beside_data
    combined_data = pd.concat([specific_data, beside_data])
    
    # Apply combined_data to convert_label function
    new_data = combined_data.apply(encode_label, label=label, axis=1)
    
    # Reset index and shuffle
    return new_data.sample(frac=1).reset_index(drop=True)

In [9]:
print('First 5 observations of data: \n')
take_200(crop_data, 'papaya').head(5)

First 5 observations of data: 



,temperature,humidity,rainfall,label
0,35.893075,82.668507,66.539466,0
1,24.872121,93.905601,135.169452,1
2,43.080227,93.903057,211.852906,1
3,21.377847,92.720437,106.141702,0
4,18.397591,36.826393,93.123306,0


## Build the Model

In [15]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [20]:
# Create model for every label-dataset
def model(data, labels):
    for label in labels:

        # Initiate data
        crop_data = take_200(data, label)
        
        # Initiate variables
        X = crop_data
        y = X.pop('label')
        
        # Split Training and Testing Dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Perform Standard Scaling
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)

        # Initiate a model structure
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, input_shape=[3], activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
             loss='binary_crossentropy',
             metrics=['accuracy'])
        
        # Fitting the Model
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)

        # Save and convert the model
        export_dir = '/tmp/model-' + label + '/'
        tf.saved_model.save(model, export_dir=export_dir)

        converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

        # Set the optimzations
        # converter.optimizations = [optimization]

        tflite_model = converter.convert()
        path = '/tmp/model-' + label + '/' + label + '-model.tflite'
        tflite_model_file = pathlib.Path(path)
        
        print('{} model => Accuracy: {:.2%} & Validation Accuracy: {:.2%}\n'.format(label.capitalize(), history.history['accuracy'][-1], 
                                                                          history.history['val_accuracy'][-1]))

In [21]:
model(crop_data, first_11_labels)

INFO:tensorflow:Assets written to: /tmp/model-rice/assets


Rice model => Accuracy: 100.00% & Validation Accuracy: 95.00%

INFO:tensorflow:Assets written to: /tmp/model-maize/assets


INFO:tensorflow:Assets written to: /tmp/model-maize/assets


Maize model => Accuracy: 100.00% & Validation Accuracy: 97.50%

INFO:tensorflow:Assets written to: /tmp/model-chickpea/assets


INFO:tensorflow:Assets written to: /tmp/model-chickpea/assets


...

Lentil model => Accuracy: 98.75% & Validation Accuracy: 100.00%

INFO:tensorflow:Assets written to: /tmp/model-pomegranate/assets


INFO:tensorflow:Assets written to: /tmp/model-pomegranate/assets


Pomegranate model => Accuracy: 95.63% & Validation Accuracy: 95.00%

INFO:tensorflow:Assets written to: /tmp/model-banana/assets


INFO:tensorflow:Assets written to: /tmp/model-banana/assets


Banana model => Accuracy: 100.00% & Validation Accuracy: 100.00%

